In [1]:
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-g0dz0wre
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-g0dz0wre
     |████████████████████████████████| 132 kB 9.5 MB/s 
     |████████████████████████████████| 344 kB 38.5 MB/s 
     |████████████████████████████████| 47.3 MB 66 kB/s 
     |████████████████████████████████| 4.5 MB 31.0 MB/s 
     |████████████████████████████████| 1.2 MB 37.9 MB/s 
     |████████████████████████████████| 4.0 MB 41.9 MB/s 
     |████████████████████████████████| 596 kB 10.0 MB/s 
     |████████████████████████████████| 77 kB 2.7 MB/s 
     |████████████████████████████████| 6.6 MB 16.1 MB/s 
     |████████████████████████████████| 895 kB 13.2 MB/s 
     |████████████████████████████████| 8.7 MB 35.3 MB/s 
     |████████████████████████████████| 79 kB 6.8 MB/s 
     |████████████████████████████████| 127 kB 41.9 MB/s 
  Created wheel for kobert: filename=kobe

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [3]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [4]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
device = torch.device("cuda:0")

In [6]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [7]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [8]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [9]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
epochs = 300
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [11]:
import pandas as pd

PATH = '/content/gdrive/MyDrive/Data/hate_word/'
category = '성별'

df = pd.read_csv(PATH+f'{category}.csv')

In [12]:
from sklearn.model_selection import train_test_split

def dataSplit(dataset, y_label):
  train, test = train_test_split(dataset, test_size = 0.2, stratify = dataset[y_label], random_state =427)
  train = train[['문장', y_label]]
  test = test[['문장', y_label]]
  return train, test

In [13]:
train, test = dataSplit(df, category)

# validation 추가
train, val = dataSplit(train, category)

In [14]:
train.to_csv('./df_train.tsv', sep='\t', encoding='utf-8', index=False)
test.to_csv('./df_test.tsv', sep='\t', encoding='utf-8', index=False)
val.to_csv('./df_val.tsv', sep='\t', encoding='utf-8', index=False)

In [15]:
dataset_train = nlp.data.TSVDataset("./df_train.tsv", field_indices=[0,1], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('./df_test.tsv', field_indices=[0,1], num_discard_samples=1)
dataset_val = nlp.data.TSVDataset('./df_val.tsv', field_indices=[0,1], num_discard_samples=1)

In [16]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)
data_val = BERTDataset(dataset_val, 0, 1, tok, max_len, True, False)

In [17]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size)
val_dataloader = torch.utils.data.DataLoader(data_val, batch_size=batch_size)

In [21]:
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size = 768, num_classes=2, dr_rate=None, params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [22]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [23]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [25]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
#loss_fn = nn.BCEWithLogitsLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [26]:
t_total = len(train_dataloader) * epochs
warmup_step = int(t_total * warmup_ratio)

In [27]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [28]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [29]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path=f'/content/gdrive/MyDrive/Data/hate_word/checkpoint_{category}.pt'):
        
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [30]:
early_stopping = EarlyStopping(patience = 7, verbose = True)

In [ ]:
from sklearn.metrics import roc_auc_score

for e in range(epochs):
    train_loss_list = [] 
    val_loss_list = []
    val_score_list = []

    epoch_train_loss = []
    epoch_val_loss = []
    epoch_val_score = []

    model.train()

    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_loss_list.append(loss.item())

        
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(val_dataloader), total=len(val_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        val_score = roc_auc_score(label.tolist(), out[:,1].tolist())
        val_loss_list.append(loss.item())
        val_score_list.append(val_score)
    
    train_loss = np.average(train_loss_list)
    val_loss = np.average(val_loss_list)
    val_score = np.average(val_score_list)

    epoch_train_loss.append(train_loss)
    epoch_val_loss.append(val_loss)
    epoch_val_score.append(val_score)
    epoch_len = len(str(epochs))

    print_msg = (f'[{e:>{epoch_len}}/{epochs:>{epoch_len}}] ' +
                 f'train_loss: {train_loss:.5f} ' +
                 f'valid_loss: {val_loss:.5f} ' +
                 f'valid_score: {val_score:.5f}')

    print(print_msg)

    # clear lists to track next epoch
    train_loss_list = []
    val_loss_list = []
    val_score_list = []
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print('early stopping')
        break
  

  0%|          | 0/73 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

[  0/300] train_loss: 0.71569 valid_loss: 0.68723 valid_score: 0.51888
Validation loss decreased (inf --> 0.687231).  Saving model ...


  0%|          | 0/73 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

...

[ 11/300] train_loss: 0.08440 valid_loss: 0.34188 valid_score: 0.96272
EarlyStopping counter: 5 out of 7


  0%|          | 0/73 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

[ 12/300] train_loss: 0.09623 valid_loss: 0.32235 valid_score: 0.96970
EarlyStopping counter: 6 out of 7


  0%|          | 0/73 [00:00<?, ?it/s]

In [ ]:
model.load_state_dict(torch.load(f'/content/gdrive/MyDrive/Data/hate_word/checkpoint_{category}.pt'))   